In [1]:
import os
import pandas as pd
import pickle

from dotenv import load_dotenv
from routingGA import RoutingGA
from domain.vehicle import Vehicle
from domain.stop import Stop
from abi.libs.rust_solver_lib import GAParameters
from abi.libs.cpp_solver_lib import GALib
from abi.libs.rust_solver_lib import RustSolverLib
from data.stops import stops
from distancesAPI import MapsAPI

%reload_ext autoreload
%autoreload 2

load_dotenv(override=True)

True

In [2]:
refetch_distances = False
calculate_time = False
plot_markers = False

In [3]:
addresses = [x.address for x in stops]
mapsAPI = MapsAPI(os.getenv('API_KEY'))
address_chunks = MapsAPI.split_in_chunks(addresses, 25)
distances = mapsAPI.split_distance_request(address_chunks, calculateTime = calculate_time) if refetch_distances else MapsAPI.get_from_cache()
distances = MapsAPI.convert_to_symmetric(distances)

In [4]:
df = pd.DataFrame(distances)

In [5]:
with open('cache/distance_matrix', 'rb') as f:
    best_ever = pickle.load(f)

In [6]:
def insert_ids_in_distances(distances):
    return [[src_id, dest_id, distance] for src_id, sublist in enumerate(distances) for dest_id, distance in enumerate(sublist)]


In [30]:
distances_with_ids = insert_ids_in_distances(distances)
vehicles = [Vehicle(0, 1000), Vehicle(1, 1000), Vehicle(2, 1000), Vehicle(3, 1000)]
parameters = GAParameters(population_size=200, elite_size=3, mutation_rate=0.05, max_crossover_tries=10, max_generations=800)
rust_lib = RustSolverLib(os.getenv('RUST_SOLVER_LIB'))
rust_result = rust_lib.run_genetic_solver(vehicles[:2], stops, distances_with_ids, parameters)
print(*rust_result)

0 31 28 22 17 18 23 25 24 26 19 16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


Bad pipe message: %s [b'Y\rNn\xc7~\xae\x1b\x8d\xce\x93\x1e\xa2\xb9\xdb\xc5\xc3\xdb \xc8\xf1W\xc8\xe0\xf3"\xec{\xb8\xe5\x9a\xa6\x85\x92\x9a\x90T\xdd\tP\t\x1a\x90\x1b\x16lh"\xe6C\x03\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 ~\xb6\x91\xd9\xa4[\xff\xe7\xaa\xf4n\x0c\xc6]\x99\x19I\x1a\xd3Flm\x81\x13\x0c\xf0\xfe']
Bad pipe message: %s [b'\xfb\xa1\xees\x11\xe9)\xd6\xd8^<\xf3\xbd\x90B\xc8=\xb0 \xa4w\x960\xc8\xc6\xc5\x1b\xb6$1\x8b\x9fv\xda@\xday\xd5zq"\xc9\xec\xc8\x87\xc4\x98']
Bad pipe message: %s [b'\xaf\xc9\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0

In [8]:
routingGA = RoutingGA(popSize=200, qtyLocations=len(distances) - 1, qtyRoutes=3,
                          maxGenerations=800, selectionK=5, mutationRate=0.05, optRate=0.02,distances=distances)
                           
lib = GALib(routingGA=routingGA,
            libPath=os.getenv('CPP_SOLVER_LIB'))

In [9]:
result = lib.run()
print(routingGA.calculate_total_distance(result))
print([len(x) for x in result])
result

7459
[11, 19, 3, 3]


[[0, 16, 18, 29, 2, 13, 4, 10, 12, 19, 24],
 [0, 14, 15, 27, 32, 20, 8, 5, 22, 9, 26, 11, 7, 3, 30, 25, 6, 23, 21],
 [0, 31, 17],
 [0, 28, 1]]

In [10]:
best_n2 = [[0, 15, 26, 9, 23, 19, 28, 31, 21, 2, 25, 32, 22, 17, 27, 18, 10, 14],
 [0, 11, 6, 8, 5, 7, 16, 12, 4, 20, 3, 30, 29, 24, 13, 1]]

In [11]:
best_n3 = [[0, 29, 31, 19, 28, 15, 23, 30, 24, 22, 18],
 [0, 3, 16, 21, 14, 13, 10, 27, 25, 6, 7, 9, 17, 8, 20, 2, 11, 1, 12],
 [0, 5, 26, 4, 32]]

In [12]:
best_n4 = [[0, 30, 29, 31],
 [0, 11, 15, 12, 14, 5, 22, 1, 13],
 [0, 4, 3, 28, 2, 20, 19, 17, 27, 25, 24, 10, 8, 7, 9, 6, 32, 26, 16, 21, 18, 23]]

In [13]:
routingGA.calculate_total_distance(result)

7459

In [14]:
# with open('cache/best_ever', 'wb'):
#     pickle.dumps(best)

In [15]:
# route_addresses = [[addresses[y] for y in x] for x in best]
# results = mapsAPI.split_directions_request(route_addresses[0])
# mapsAPI.save_to_image(results, f"route_test")

In [16]:
# route_addresses = [[addresses[y] for y in x] for x in best_n4]
# imageCount = 1
# for route in route_addresses:
#     directions = mapsAPI.directions(route[0], route[0], route[1: 25])
#     mapsAPI.save_to_image(directions, f"n4_route_{imageCount}")
#     imageCount += 1

In [17]:
if plot_markers:
    plot = mapsAPI.plot_markers(addresses)
    with open('test.jpg', 'wb') as f:
        for chunk in plot:
            f.write(chunk)